In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.tree_node import NodeRef
from evaluator import BenchViewer, Evaluator
from debugger.debugger import Debugger3
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files, get_name_hash

In [ ]:
qs = UNSTABLE_MARIPOSA
# qs = UNSTABLE_VERUS
mv = BenchViewer(qs)
# for q in qs:
#     print(q)
#     rev = Evaluator(q)

In [3]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     487 | 89.36 %      |
| DebuggerStatus.SINGLETON_NOT_RAN              |      36 | 6.61 %       |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |       9 | 1.65 %       |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |       9 | 1.65 %       |
| DebuggerStatus.NO_PROOF                       |       4 | 0.73 %       |
| total                                         |     545 | 100.00 %     |


In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print(q)

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print("./src/debugger3.py --create-singleton -i", q)

for q in mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED]:
    print("./src/debugger3.py --create-singleton -i", q)

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
# for q in mv.status[DebuggerStatus.SINGLETON_TESTED_EMPTY]:
# for q in mv.status[DebuggerStatus.FILTERED_NOT_RAN]:
    assert len(list_smt2_files(mv[q].singleton_dir)) != 0
    print("./src/exper_wizard.py manager -e verify --total-parts 12 -i ", mv[q].singleton_dir, "--clear")
    print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
    print("python3 src/carve_and_rerun.py", mv[q].filtered_dir)

In [ ]:
print(len(mv.fixable)/len(mv.status[DebuggerStatus.SINGLETON_FINISHED]))

# import numpy as np
# bad_min_ranks = dict()
# min_ranks = []

# for q in mv.fixable:
#     r = mv.reviewers[q]
#     report = r.build_report()
#     indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
#     report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
#     min_rank = report.freq.loc[indices]["rank"].min()
#     max_rank = report.freq.loc[indices]["rank"].max()
#     min_ranks.append(min_rank)
#     if min_rank <= 10:
#         continue
#     bad_min_ranks[q] = (min_rank, len(indices))

# min_ranks = np.array(min_ranks)

# for q, (rank, feasible) in bad_min_ranks.items():
#     print(int(rank), feasible, q)

In [ ]:
def fmt_ratio(x, y):
    return f"({round(x / y * 100, 1)}%)"

def fmt_ratio_2(x, y):
    return f"{x}/{y} ({round(x / y * 100, 1)}%)"

def count_leq(r, n):
    return np.where(r <= n)[0].shape[0]

fixable_count = len(mv.fixable)
finished_count = len(mv.status[DebuggerStatus.SINGLETON_FINISHED])
bench_total = mv.status.total
print("finished", fmt_ratio_2(finished_count, bench_total))
print("\tfixable", fmt_ratio_2(fixable_count, finished_count))
rcount = count_leq(min_ranks, 1)
print("\t\t", "top-1 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 3)
print("\t\t", "top-3 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 10)
print("\t\t", "top-10", fmt_ratio_2(rcount, fixable_count))
print("\tno-fixes", fmt_ratio_2(finished_count - fixable_count, finished_count))
no_proof = len(mv.status[DebuggerStatus.NO_PROOF])
print("no-proof", fmt_ratio_2(no_proof, bench_total))
parse_fail = len(mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]) + len(mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED])
print("parse-fail", fmt_ratio_2(parse_fail, bench_total))
in_progress = bench_total - finished_count - parse_fail - no_proof
print("in-progress", fmt_ratio_2(in_progress, bench_total))

In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
dirs_to_send = []
additional = ""

for q in mv.fixable:
    r = mv[q]
    if r.name_hash in LIKELY_FIXED:
        continue
    r.collect_garbage()
    dirs_to_send += r.get_dirs_to_sync()
    additional += f"\"{r.name_hash}\",\n"

print(additional)

for i, d in enumerate(dirs_to_send):
    if i % 5 == 0:
        print("echo '", (i //5 +1), "/", len(dirs_to_send)//5, "'")
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")

In [7]:
from utils.analysis_utils import Categorizer

f_modes = Categorizer()

for query in mv.unfixable | mv.status[DebuggerStatus.SINGLETON_NOT_RAN].items:
    rev: Evaluator = mv[query]
    mi = rev.get_trace_info()
    reason = mi.get_failed_reason()
    f_modes.add_item(reason, query)

f_modes.finalize()

In [8]:
f_modes.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| TraceFailure.TIMEOUT      |     114 | 82.61 %      |
| TraceFailure.FAST_UNKNOWN |      21 | 15.22 %      |
| TraceFailure.SLOW_UNKNOWN |       3 | 2.17 %       |
| total                     |     138 | 100.00 %     |


In [10]:
from debugger.mutant_info import TraceFailure

for q in f_modes[TraceFailure.TIMEOUT]:
    print(q)

data/projs/bench_unstable/base.z3/d_komodo--verified-map_common.gen.dfyImpl___module.__default.va__lemma__init__secure__page.smt2
data/projs/bench_unstable/base.z3/d_lvbkv--lib-Buckets-PackedKV.i.dfy.Impl__DynamicPkv.__default.MergeToChildrenIter.smt2
data/projs/bench_unstable/base.z3/d_lvbkv--MapSpec-TSJ.i.dfy.Impl__TSJ.__default.Move2to3StepPreservesInv.smt2
data/projs/bench_unstable/base.z3/d_komodo--verified-init_dispatcher.gen.dfyImpl___module.__default.va__lemma__init__dispatcher__page__zero.smt2
data/projs/bench_unstable/base.z3/d_komodo--verified-allocate_page.gen.dfyImpl___module.__default.va__lemma__allocate__page__success.smt2
data/projs/bench_unstable/base.z3/d_komodo--verified-ptebits.i.dfyImpl___module.__default.lemma__ARM__L2PTE__helper_split1.smt2
data/projs/bench_unstable/base.z3/d_lvbkv--lib-DataStructures-BtreeModel.i.dfy.Impl__BtreeModel.__default.InsertLeafIsCorrect.smt2
data/projs/bench_unstable/base.z3/d_lvbkv--Impl-NodeImpl.i.dfy.Impl__NodeImpl.Node.CutoffNodeAn

In [14]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED].items | mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED].items:
    print(q)

data/projs/bench_unstable/base.z3/fs_dice--queries-L0.Impl.Certificate-16.smt2
data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.ReqWrite2StepPreservesInv.smt2
data/projs/bench_unstable/base.z3/fs_dice--queries-L0.Impl.Certificate-14.smt2
data/projs/bench_unstable/base.z3/fs_dice--queries-L0.X509.DeviceIDCRI-18.smt2
data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.RespReadStepPreservesInv.smt2
data/projs/bench_unstable/base.z3/d_lvbkv--Impl-QueryImpl.i.dfy.Impl__QueryImpl.__default.queryIterate.smt2
data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.ReqWriteStepPreservesInv.smt2
data/projs/bench_unstable/base.z3/fs_dice--queries-L0.Impl.Certificate-12.smt2
data/projs/bench_unstable/base.z3/d_fvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.Impl__ByteSystem.__default.RespWriteStepPreservesInv.smt2
data/projs/bench_unsta

In [ ]:
from debugger.mutant_info import TraceFailure

def double_check_tested(ev: Evaluator):
    # mi = ev.get_trace_info()
    # trace_size = mi.get_trace_size()
    # if trace_size > 300 * 1024 * 1024:
    #     return (trace_size, ev.name_hash, mi.trace_path)
    actions = ev.editor.get_singleton_actions()
    expected = len(actions)
    tested = len(ev.build_report().tested)
    if tested != expected:
        return (expected, tested, ev.name_hash)
    return None

# for q in f_modes[TraceFailure.TIMEOUT]:
#     ev = mv[q]

from multiprocessing import Pool
pool = Pool(6)

res = pool.map(double_check_tested, [mv[q] for q in mv.unfixable])
res = [r for r in res if r is not None]

In [ ]:
# count = 0
# for (actions, tested, hash) in res:
#     if len(actions) <= tested:
#         continue
#     print(f"echo {count}/61")
#     print("rm -r data/projs/singleton_" + hash)
#     print("rm -r data/dbs/singleton_" + hash)
#     print("rm -r data/projs/singleton_" + hash + ".filtered")
#     print("rm -r data/dbs/singleton_" + hash + ".filtered")
#     print("rm ./cache/" + hash + ".report")
#     print("rm dbg/" + hash + "/edits.json")
#     print("./src/debugger3.py --create-singleton -i", hash)
#     count += 1
